In [5]:
import datetime
import json
import os
import requests
import datetime

# replace with '<YOUR-DARKSKY_API-KEY>'
DARKSKY_API_KEY = '4eb053f74d7f5f150083e339c7cb325c'

boroughs_dict = {"usaquen": "4.719722,-74.036667","chapinero": "4.645833,-74.063333","santafe": "4.613889,-74.078611",
"sancristobal": "4.564722,-74.083333","usme": "4.446389,-74.152222","tunjuelito": "4.582222,-74.131944",
"bosa": "4.616944,-74.19","kennedy": "4.643611,-74.153333","fontibon": "4.678611,-74.141111",
"engativa": "4.726111,-74.1","suba": "4.741,-74.084","barriosunidos": "4.678611,-74.078611",
"teusaquillo": "4.670556,-74.093056","losmartires": "4.604444,-74.09","antonionariño": "4.591655,-74.106505",
"puentearanda": "4.6125,-74.106667","lacandelaria": "4.591722,-74.074131","rafaeluribe": "4.579722,-74.1175",
"ciudadbolivar": "4.536111,-74.138889","sumapaz": "4.26,-74.178333"}

coordinates_list = list(boroughs_dict.values())

In [2]:
data_folder = '../data/darksky/data'
if not os.path.isdir(data_folder):
    os.makedirs(data_folder)

In [20]:
def date_range(start:datetime.date, end:datetime.date):
    r = (end+datetime.timedelta(days=1)-start).days
    return [str(start+datetime.timedelta(days=i)) + "T00:00:00Z" for i in range(r)]
start = datetime.date(2020,4,13)
end = datetime.date(2020,4,17)
date_list = date_range(start, end)

In [56]:
# Generate the URL
def generate_url(coordinates:str, timestamp:str):
    url = f"https://api.darksky.net/forecast/{DARKSKY_API_KEY}/{coordinates},{timestamp}?units=si&exclude=flags,alerts"
    return url

test_url = generate_url(coordinates_list[0],date_list[0])

In [57]:
def fetch_request(url:str):
    response = requests.get(url)
    weather_data = response.json()
    return weather_data

In [58]:
def save_json(weather_json):
    current_datetime = datetime.datetime.fromtimestamp(weather_json['currently']['time']).strftime('%Y_%m_%d')
    filename = os.path.join(data_folder, f'weather_{current_datetime}.json')
    with open(filename, 'w+') as f:
        f.write(json.dumps(weather_json))

In [59]:
save_json(fetch_request(test_url))

In [71]:
import datetime
import json
import pandas as pd
import glob

def save_data_to_csv():
    df = pd.DataFrame()
    for json_file in glob.glob( data_folder + '/*.json'):
        filename = os.path.basename(json_file).split('.')[0] + '.csv'
        with open(json_file) as f:
            data = json.load(f)
        df_file = pd.DataFrame(data['hourly']['data'])
        # Convert time into a proper datetime object
        df = df.append(df_file)
        df['time'] = pd.to_datetime(df['time'], unit='s')

        output_folder_name = '/..data/darksky/output_data'
        if not os.path.isdir(output_folder_name):
            os.makedirs(output_folder_name)
        filename = os.path.join(output_folder_name, filename)
        df.to_csv(filename, index=False)

In [72]:
save_data_to_csv()